## Recipe finder
### by Abilash Ramesh

### Load all packages

In [6]:
import pandas as pd
import numpy as np
import json
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


### Load recipe dataset

In [7]:
def get_data(filename):
    df = pd.read_json(filename)
    new = []
    for s in df['ingredients']:
        s = ' '.join(s)
        new.append(s)
    df['ing'] = new
    return df


### Get ingredients from user

In [8]:
def get_user_ing(string, df):
    user_ing = string
    df = df.append({'ing':user_ing}, ignore_index=True)
    return df


### Train model and vectorizer

In [9]:
def train_data(df):
    vectorizer = TfidfVectorizer(use_idf = True, smooth_idf=True, stop_words = 'english',max_features = 4000)
    ing_vect = vectorizer.fit_transform((df['ing'].values))
    vec = ing_vect.todense()
    X_train_df = vec[:-1]
    y_train_df = df['cuisine'][:-1]
    X_test_df = vec[-1]
    n = 8
    model = KNeighborsClassifier(n_neighbors = n, weights='uniform', algorithm='auto', metric='minkowski')
    preds = model.fit(X_train_df,y_train_df)
    return X_test_df, model


### Obtain results for the given set of ingredients

In [10]:
def get_results(test, model):
    
    predicted_class = model.classes_
    predicted_single_cuisine = model.predict(test)
    predicted_cuisine = model.predict_proba(test)[0]
    match_perc,match_id = model.kneighbors(test)
    pos = np.where(predicted_class == predicted_single_cuisine)
    print ("The model predicts that the ingredients resembles %s (%f resemblence)" %(predicted_single_cuisine[0], predicted_cuisine[pos]*100))
    for i in range(len(match_id[0])):
        print ('Recipe No: %d (%f probable match)'%(match_id[0][i], match_perc[0][i]))


### The recipes and the closest type of cuisine is displayed
You will need to run the below code every time you want to find the cuisine/recipes

In [16]:
df = get_data('yummly.json')
df = get_user_ing('feta cheese, romaine lettuce', df)
test, mod = train_data(df)

In [17]:
get_results(test, mod)

The model predicts that the ingredients resembles greek (75.000000 resemblence)
Recipe No: 34721 (0.780192 probable match)
Recipe No: 18509 (0.791102 probable match)
Recipe No: 2394 (0.920194 probable match)
Recipe No: 0 (0.949259 probable match)
Recipe No: 22127 (0.956448 probable match)
Recipe No: 37795 (0.960988 probable match)
Recipe No: 8992 (0.974346 probable match)
Recipe No: 37955 (0.992854 probable match)
